<a href="https://colab.research.google.com/github/pcjo1202/Traffic_Manager/blob/main/Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch.nn.init
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


학습에 사용할 파라미터 설정.

In [ ]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [ ]:
data_train = '/content/drive/MyDrive/CNN/train'  # 데이터가 있는 경로
data_test = '/content/drive/My Drive/CNN/test'

# 데이터셋에 적용할 변환 정의
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),  # 텐서로 변환
])

%cd /content/drive/MyDrive/CNN
!unzip -qq "/content/drive/MyDrive/CNN/train.zip"

데이터셋 불러오기

In [ ]:
# ImageFolder를 사용하여 train 데이터 불러오기
train_dataset = dsets.ImageFolder(root=data_train, transform=transform)

# ImageFolder를 사용하여 test 데이터 불러오기
test_dataset = dsets.ImageFolder(root=data_test, transform=transform)

데이터로더를 사용하여 배치 크기를 지정

In [ ]:
# 데이터로더 생성
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

import torch
import torch.nn as nn
# 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(100, 3, 224, 224)
print('텐서의 크기 : {}'.format(inputs.shape))
-텐서의 크기 : torch.Size([100, 3, 224, 224])
conv1 = nn.Conv2d(3, 16, 3, padding=1)
print(conv1)
-Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
print(conv2)
output
Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
pool = nn.MaxPool2d(2)
print(pool)
-MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
out = conv1(inputs)
print(out.shape)
-torch.Size([100, 16, 224, 224])
out = pool(out)
print(out.shape)
-torch.Size([100, 16, 112, 112])
out = conv2(out)
print(out.shape)
-torch.Size([100, 32, 112, 112])
out = pool(out)
print(out.shape)
-torch.Size([100, 32, 56, 56])

out.size(0)
-100
out.size(1)
-32
out.size(2)
-56
out.size(3)
-56
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1)
print(out.shape)
-torch.Size([100, 100352])
fc = nn.Linear(100352, 100) # input_dim = 3,136, output_dim = 10
out = fc(out)
print(out.shape)
-torch.Size([100, 100])

클래스로 모델을 설계

In [ ]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        # L1 ImgIn shape=(?, 224, 224, 3)
        #    Conv     -> (?, 224, 224, 16)
        #    Pool     -> (?, 112, 112, 16)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)) # 이미지 크기 절반

        # L2 ImgIn shape=(?, 112, 112, 16)
        #    Conv     -> (?, 112, 112, 32)
        #    Pool     -> (?, 56, 56, 32)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # L3 ImgIn shape=(?, 56, 56, 32)
        #    Conv     -> (?, 56, 56, 64)
        #    Pool     -> (?, 28, 28, 64)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        self.fc1 = torch.nn.Linear( 9 *  9 * 32, 100, bias=True)
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))

        # L5 Final FC 100 inputs -> 100 outputs
        self.fc2 = torch.nn.Linear(100, 100, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.layer4(out)
        out = self.fc2(out)
        return out

모델 정의

In [ ]:
# CNN 모델 정의
model = CNN().to(device)

#비용 함수와 옵티마이저를 정의
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#총 배치의 수 출력
total_batch = len(train_dataloader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 134


총 배치의 수는 134입니다. 그런데 배치 크기를 100으로 했으므로 결국 훈련 데이터는 총 13,400개란 의미입니다. 이제 모델을 훈련시켜보겠습니다.

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_dataloader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 4.5583272
[Epoch:    2] cost = 4.26724863
[Epoch:    3] cost = 4.04080772
[Epoch:    4] cost = 3.89961195
[Epoch:    5] cost = 3.7592175
[Epoch:    6] cost = 3.65498805
[Epoch:    7] cost = 3.56467652
[Epoch:    8] cost = 3.50953865
[Epoch:    9] cost = 3.44234252
[Epoch:   10] cost = 3.36996412
[Epoch:   11] cost = 3.30972743
[Epoch:   12] cost = 3.26448655
[Epoch:   13] cost = 3.24501228
[Epoch:   14] cost = 3.18759608
[Epoch:   15] cost = 3.14789701


In [ ]:
# 데이터셋에 적용할 변환 정의
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),  # 텐서로 변환
])

# ImageFolder를 사용하여 train 데이터 불러오기
train_dataset = dsets.ImageFolder(root=data_train, transform=transform)

# 클래스 확인
classes = train_dataset.classes

# 클래스 이름과 인덱스 매핑
class_to_idx = train_dataset.class_to_idx

# 클래스 이름 및 해당 인덱스 출력
for k, v in class_to_idx.items():
    print(f"Class: {k}, Index: {v}")

Class: air hockey, Index: 0
Class: ampute football, Index: 1
Class: archery, Index: 2
Class: arm wrestling, Index: 3
Class: axe throwing, Index: 4
Class: balance beam, Index: 5
Class: barell racing, Index: 6
Class: baseball, Index: 7
Class: basketball, Index: 8
Class: baton twirling, Index: 9
Class: bike polo, Index: 10
Class: billiards, Index: 11
Class: bmx, Index: 12
Class: bobsled, Index: 13
Class: bowling, Index: 14
Class: boxing, Index: 15
Class: bull riding, Index: 16
Class: bungee jumping, Index: 17
Class: canoe slamon, Index: 18
Class: cheerleading, Index: 19
Class: chuckwagon racing, Index: 20
Class: cricket, Index: 21
Class: croquet, Index: 22
Class: curling, Index: 23
Class: disc golf, Index: 24
Class: fencing, Index: 25
Class: field hockey, Index: 26
Class: figure skating men, Index: 27
Class: figure skating pairs, Index: 28
Class: figure skating women, Index: 29
Class: fly fishing, Index: 30
Class: football, Index: 31
Class: formula 1 racing, Index: 32
Class: frisbee, Inde

테스트

In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = []
    Y_test = []
    for data, labels in test_dataloader:
        data = data.to(device)
        label = labels.to(device)
        X_test.append(data)
        Y_test.append(label)

    X_test = torch.cat(X_test, dim=0)
    Y_test = torch.cat(Y_test, dim=0)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.1940000057220459


층을 깊게 쌓는 것이 정확도를 올려주진 않음. 효율적으로 쌓는 것도 중요.